# Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from pyspark.sql.functions import length

## Initialize spark context

In [2]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "1")
         .config("spark.executor.memory", "12g")
         .config("spark.executor.cores", "2")
         .config("spark.driver.memory", "10g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "64g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .config("spark.sql.legacy.timeParserPolicy", "LEGACY")
         .config("spark.sql.legacy.parquet.int96RebaseModeInWrite", "CORRECTED")
         .getOrCreate())


sc = spark.sparkContext
sc.addFile('/home/jovyan/Work/ej/paquetes/nlppen/nlppen.zip')
sc.uiWebUrl

21/11/04 20:08:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


'http://69abb479a5e1:4040'

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [3]:

anno_inf = 2010
anno_sup = 2021

terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}


seleccion = Seleccion(terminos, spark, parquet_path='../../src/datasets/complete', datasets_path='./datasets/estructurales', filtro=f'anno >= {anno_inf} AND anno < {anno_sup}')
seleccion.filtrar_sentencias(preprocess=solo_portanto, keepRowEmpty=True, partition_cols=['anno'], precargar=False)

estructurales = SentenciasEstructurales(seleccion)

seleccion.sdf.count()

21/11/04 20:09:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


151269

In [5]:
spark.read.parquet('../../src/datasets/complete').filter('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ').count()

96773

# Formar dataset de sentencias estructurales

In [1]:
from pyspark.sql.types import *

columnas = {
    'se ordena PER' : ArrayType(StringType()),
    'se ordena LOC' : ArrayType(StringType()),
    'se ordena ORG' : ArrayType(StringType()),
    'se ordena MISC' : ArrayType(StringType()),
    'se ordena GPE' : ArrayType(StringType()),
    'se ordena Ent Pub' : ArrayType(StringType())
}
estructurales.separarSeOrdena(columnas, True, True)

columnas = {
    'extension sentencia' : IntegerType(),
    'extension por lo tanto' : IntegerType()
}
estructurales.extraerExtension(columnas, True)

columnas = {
    'plazosDefinidos' : ArrayType(TimestampType())
}
estructurales.plazosDefinidos(columnas, True)

columnas = {
    'FechaSolicitud' : TimestampType(),
}

estructurales.extrarFechaRecibido(columnas, True)

columnas = {
    'num resolucion' : StringType()
}

estructurales.extraerNumeroSentencia(columnas, True)

In [5]:
from pyspark.sql.types import *
columnas = {
    'inst internacionales' : ArrayType(StringType())
}

estructurales.extraerInstrumentosInternacionales(columnas, True)

columnas = {
    'derechos Norm' : ArrayType(StringType()),
    'derechos GenXPat' : ArrayType(StringType())
}
estructurales.extraerDerechos(columnas, True)

columnas = {
    'derechos Acotados' : ArrayType(StringType()),
    'derechos General' : ArrayType(StringType()),
    'derechos Fundamental' : ArrayType(StringType()),
    'derechos Humano' : ArrayType(StringType())
}

estructurales.extraerDerechosSinNormalizar(columnas, True)

columnas = {
        'citasFechas' : MapType(StringType(), TimestampType())
    }
    estructurales.extrarCitaSentenciasFecha(columnas, True)
#Sobreescribir el dataset de filtro de sentencias con las nuevas columnas

DataFrame[archivo: string, txt: string, cleanText: string, num: bigint, fechahora_ext: timestamp, redactor_ext: string, tipoResolucion_ext: string, termino_ext: string, voto_salvado: boolean, tema: string, Recurrente_ents: array<string>, Cantón_ents: array<string>, Fecha_ents: array<string>, Constitución_ents: array<bigint>, Redactor_ents: array<string>, Ley_ents: array<string>, EntidadPublica_ents: array<string>, tema_prob: double, termino_lst: array<string>, expediente: string, Fechahora_ents: array<string>, CitaSentencias_ents: array<string>, Recurrido_ents: array<string>, Reglamento_ents: array<string>, Magistrado_ents: array<string>, _id: string, anno: int, seguimiento: int, se_ordena: int, plan: int, plazo: int, inst_internacionales: array<string>, derechos_Norm: array<string>, derechos_GenXPat: array<string>, derechos_Acotados: array<string>, derechos_General: array<string>, derechos_Fundamental: array<string>, derechos_Humano: array<string>]

In [6]:
estructurales.seleccion.guardarDatos(parquet_file='extracciones2', partition_cols=['anno'], borrar=False)

21/11/04 20:12:49 WARN BlockManager: Putting block rdd_45_10 failed due to exception org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/jovyan/Work/ej/paquetes/nlppen/nlppen/spacy_internationals.py", line 130, in extractInternational
    nlp
NameError: name 'nlp' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jovyan/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/jovyan/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/jovyan/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/jovyan/spark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/home/jovyan/Work/ej/paquetes/nlppen/nlppen/sentencias_estr

Py4JJavaError: An error occurred while calling o152.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apac

21/11/04 20:12:52 WARN PythonRunner: Incomplete task 6.0 in stage 6 (TID 294) interrupted: Attempting to kill Python Worker
21/11/04 20:12:52 WARN PythonRunner: Incomplete task 1.0 in stage 6 (TID 289) interrupted: Attempting to kill Python Worker
21/11/04 20:12:52 WARN BlockManager: Putting block rdd_45_1 failed due to exception org.apache.spark.TaskKilledException.
21/11/04 20:12:52 WARN BlockManager: Putting block rdd_45_6 failed due to exception org.apache.spark.TaskKilledException.
21/11/04 20:12:52 WARN BlockManager: Block rdd_45_1 could not be removed as it was not found on disk or in memory
21/11/04 20:12:52 WARN BlockManager: Block rdd_45_6 could not be removed as it was not found on disk or in memory
21/11/04 20:12:52 WARN BlockManager: Putting block rdd_58_1 failed due to exception org.apache.spark.TaskKilledException.
21/11/04 20:12:52 WARN BlockManager: Block rdd_58_1 could not be removed as it was not found on disk or in memory
21/11/04 20:12:52 WARN BlockManager: Putting

In [ ]:
estructurales.seleccion.sdf.sample(10).toPandas()

In [ ]:
estructurales.seleccion.sdf.dropNull('FechaSolicitud').groupby('anno').count()

In [ ]:
estructurales.seleccion.sdf.select('FechaSolicitud', 'anno').filter('FechaSolicitud is not Null').toPandas().count()

In [ ]:
estructurales.seleccion.sdf.limit(15).toPandas()

# Cargar procesamiento de sentencias estructurales.

In [5]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='./datasets', datasets_path='./datasets/estructurales')
seleccion.cargarPreprocesados()

estructurales = SentenciasEstructurales(seleccion)

In [7]:
from pyspark.sql.types import *
columnas = {
    'num resolucion' : StringType()
}

estructurales.extraerNumeroSentencia(columnas, True)

21/10/21 16:45:08 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[archivo: string, txt: string, cleanText: string, num: bigint, fechahora_ext: timestamp, redactor_ext: string, tipoResolucion_ext: string, termino_ext: string, voto_salvado: boolean, tema: string, Recurrente_ents: array<string>, Cantón_ents: array<string>, Fecha_ents: array<string>, Constitución_ents: array<bigint>, Redactor_ents: array<string>, Ley_ents: array<string>, EntidadPublica_ents: array<string>, tema_prob: double, termino_lst: array<string>, expediente: string, Fechahora_ents: array<string>, CitaSentencias_ents: array<string>, Recurrido_ents: array<string>, Reglamento_ents: array<string>, Magistrado_ents: array<string>, _id: string, seguimiento: int, se_ordena: int, plan: int, plazo: int, se_ordena_PER: array<string>, se_ordena_LOC: array<string>, se_ordena_ORG: array<string>, se_ordena_MISC: array<string>, se_ordena_GPE: array<string>, se_ordena_Ent_Pub: array<string>, extension_sentencia: int, extension_por_lo_tanto: int, plazosDefinidos: array<timestamp>, FechaSol

# Zona de trabajo (ignorar)

In [ ]:
s = estructurales.seleccion.sdf
#print(solo_portanto(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'txt']))

In [ ]:
#print(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'se_ordena_PER'])

In [ ]:
#print(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'plazosDefinidos'])

In [ ]:
#print(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1])